In [1]:
import pandas as pd
import numpy as np


### Functions to pre-process data

In [2]:
def time_since_start(df):
    time_start = np.zeros(len(df))
    for i in range(len(df)):
        if i ==0 :
            time_start[i] =1
            
        else:
            if (df['mode'][i] == 'start') & (df['mode'][i-1] == 'operation') & (df['time_difference'][i] <500):
                time_start[i] = 0
                
            elif (df['mode'][i] == 'start') & (df['mode'][i-1] == 'start') & (df['time_difference'][i] <500):
                time_start[i] = time_start[i-1]+ df['time_difference'][i]
            
            elif (df['mode'][i] == 'start') & (df['time_difference'][i] >500):
                time_start[i] = 0
                
            elif (df['mode'][i] == 'operation'):
                time_start[i] = time_start[i-1]
                
    return time_start

In [3]:
def timepoint_difference(df):
    tps =np.array(df.index[:])
    tp_diff =np.diff(tps)/np.timedelta64(1, 's') 
    tp_diff = np.insert(tp_diff, 0 , 1)
    return tp_diff

In [4]:
data_2 = pd.read_parquet('input_dataset-2.parquet')
print(len(data_2))

1874087


In [5]:
print(data_2.describe())

       Unit_4_Power  Unit_4_Reactive Power  Turbine_Guide Vane Opening  \
count  1.750000e+06           1.750000e+06                1.750000e+06   
mean   2.977542e+02           5.989380e+00                9.105910e+01   
std    3.387414e+01           8.155810e+00                8.714153e+00   
min    0.000000e+00          -3.811665e+01                0.000000e+00   
25%    3.069059e+02           0.000000e+00                9.331226e+01   
50%    3.098457e+02           6.300167e+00                9.404115e+01   
75%    3.112799e+02           1.117088e+01                9.448438e+01   
max    3.233023e+02           4.910232e+01                9.618052e+01   

       Turbine_Pressure Drafttube  Turbine_Pressure Spiral Casing  \
count                1.750000e+06                    1.750000e+06   
mean                 1.577165e+02                    5.312681e+03   
std                  1.909088e+01                    2.990343e+01   
min                  1.353587e+02                    3.62

In [6]:
list(data_2.columns)

['Unit_4_Power',
 'Unit_4_Reactive Power',
 'Turbine_Guide Vane Opening',
 'Turbine_Pressure Drafttube',
 'Turbine_Pressure Spiral Casing',
 'Turbine_Rotational Speed',
 'mode',
 'Bolt_1_Steel tmp',
 'Bolt_1_Tensile',
 'Bolt_2_Tensile',
 'Bolt_3_Tensile',
 'Bolt_4_Tensile',
 'Bolt_5_Tensile',
 'Bolt_6_Tensile',
 'Bolt_1_Torsion',
 'Bolt_2_Torsion',
 'Bolt_3_Torsion',
 'Bolt_4_Torsion',
 'Bolt_5_Torsion',
 'Bolt_6_Torsion',
 'lower_bearing_vib_vrt',
 'turbine_bearing_vib_vrt']

In [7]:
#df2_op =data_2.loc[data_2['mode'] == 'operation']

In [9]:
tp_diff = timepoint_difference(data_2)
print(tp_diff)
print(tp_diff.shape)
print(np.max(tp_diff), np.min(tp_diff), np.mean(tp_diff), np.median(tp_diff), np.sort(tp_diff)[-10:])

[1. 1. 1. ... 1. 1. 1.]
(1874087,)
215516.0 1.0 1.7081944434810123 1.0 [ 41517.  45031.  49034.  51465.  73384.  80594. 142272. 147139. 154647.
 215516.]


In [10]:
new_data2 = pd.DataFrame(data_2)
new_data2.insert(0, 'time_difference', tp_diff)
print(new_data2)

                     time_difference  Unit_4_Power  Unit_4_Reactive Power  \
timepoints                                                                  
1970-12-19 09:51:44              1.0    262.204308               2.899036   
1970-12-19 09:51:45              1.0    262.104319               3.344630   
1970-12-19 09:51:46              1.0    262.004330               3.790223   
1970-12-19 09:51:47              1.0    261.904340               4.235817   
1970-12-19 09:51:48              1.0    261.804351               4.064759   
...                              ...           ...                    ...   
1971-01-25 11:06:44              1.0    308.716025               3.974309   
1971-01-25 11:06:45              1.0    308.746393               4.103262   
1971-01-25 11:06:46              1.0    308.776762               4.472929   
1971-01-25 11:06:47              1.0    308.807131               4.842597   
1971-01-25 11:06:48              1.0    308.837499               5.212264   

In [11]:
print(np.all(new_data2.loc[new_data2['mode'] == 'start', 'time_difference'] == 0))
print((new_data2.loc[new_data2['mode'] == 'operation', 'time_difference']).describe())
print((new_data2.loc[new_data2['mode'] == 'start', 'time_difference']).describe())

False
count    1.868659e+06
mean     1.000232e+00
std      1.638735e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.360000e+02
Name: time_difference, dtype: float64
count      5428.000000
mean        245.433309
std        5107.189098
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max      215516.000000
Name: time_difference, dtype: float64


In [12]:
time_start = time_since_start(new_data2)
print(time_start)

[  1.   1.   1. ... 211. 211. 211.]


In [13]:
new_data2.insert(0, 'time_start', time_start)

In [14]:
print(new_data2['time_start'].describe())

count    1.874087e+06
mean     1.668366e+02
std      8.689492e+01
min      0.000000e+00
25%      1.880000e+02
50%      2.120000e+02
75%      2.150000e+02
max      2.450000e+02
Name: time_start, dtype: float64


In [15]:
print(new_data2.loc[new_data2['mode'] == 'start' , 'time_start'].describe())
print(new_data2.loc[new_data2['mode'] == 'operation' , 'time_start'].describe())

count    5428.000000
mean      104.347089
std        61.082445
min         0.000000
25%        52.000000
50%       104.000000
75%       156.000000
max       245.000000
Name: time_start, dtype: float64
count    1.868659e+06
mean     1.670181e+02
std      8.689332e+01
min      1.000000e+00
25%      1.880000e+02
50%      2.120000e+02
75%      2.150000e+02
max      2.450000e+02
Name: time_start, dtype: float64
